# PyTorch Geometricで理解するGraph Convolutional Networks

[公式ドキュメント](https://pytorch-geometric.readthedocs.io/en/latest/index.html)

## 0. インストール
1. PyTorchのバージョンが1.2.0以上
2. torch-geometricおよび依存関係のあるパッケージをインストール

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/Graph Convolution Network"

/content/drive/My Drive/Colab Notebooks/Graph Convolution Network


In [ ]:
!python -c "import torch; print(torch.__version__)"

1.6.0+cu101


In [ ]:
! pip install --verbose --no-cache-dir torch-scatter
! pip install --verbose --no-cache-dir torch-sparse
! pip install --verbose --no-cache-dir torch-cluster
! pip install --verbose --no-cache-dir torch-spline-conv (optional)
! pip install torch-geometric

## 1. チュートリアルやってみた

### 1.1 グラフデータを扱ってみる
グラフは何かのもの同士の関係を表すのに使われるが、ものをノード、関係を表す線をエッジと呼ぶ。PyTorch Geometricではこのグラフデータは　```torch_geometric.data.Data``` で保持される。メソッドは代表的なものだけ紹介する。

* ```data.x```: ノードの特徴量行列。形状は```[ノード数, ノードの特徴量数]```
* ```data.edge_index```: COO形式のエッジ情報。形状は```[2, エッジ数]```
* ```data.edge_attr```: エッジの特徴量行列。形状は```[エッジ数, エッジの特徴量数]```
* ```data.y```: ターゲット。ノードラベルやグラフラベル。 

In [ ]:
import torch
from torch_geometric.data import Data

# ノード情報
# 3つのノード(インデックス0,1,2)があり、各ノードが特徴量-1, 0, 1をもつ。
# 形状: [ノード数, ノードの特徴量数] = [3, 1]
x = torch.tensor([[-1], [0], [1]],dtype=torch.float)

# エッジ情報
# ノード0と1が双方向で繋がっており、ノード1と2も双方向で繋がっていることを示している。
# 形状: [2, エッジ数] = [2, 4]
edge_index = torch.tensor(
    [[0, 1, 1, 2],
     [1, 0, 2, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 1])

### 1.2 代表的なデータセット
CoraやCiteseer, Pubmed, QM7, FAUSTなど代表的なデータセットを備え持つ。
ここでは酵素のENZYMESデータセットロードしてみる。
以下のコードから、ENZYMESには6種類のグラフが600個あることがわかる。
そのうちの初めのグラフは、37個のノードと168個のエッジを持つことがわかる。

In [ ]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print("グラフの数: ", len(dataset))
print("クラスの数:",dataset.num_classes)
print("ノードの特徴量の数: ", dataset.num_node_features)
print("1つめのグラフ: ", dataset[0])

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!
グラフの数:  600
クラスの数: 6
ノードの特徴量の数:  3
1つめのグラフ:  Data(edge_index=[2, 168], x=[37, 3], y=[1])


シャッフルも容易にできる。

In [ ]:
dataset = dataset.shuffle()
dataset

ENZYMES(600)

論文引用ネットワークのCoraデータセットも同様にダウンロードしてみる。(ノード(=論文)の分類問題用データセット)

In [ ]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print("グラフの数: ", len(dataset))
print("クラスの数: ", dataset.num_classes)
print("ノードの特徴量の数: ", dataset.num_node_features)
print("グラフ情報: ", dataset[0])

Processing...
Done!
グラフの数:  1
クラスの数:  7
ノードの特徴量の数:  1433
グラフ情報:  Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


ここで```Data```オブジェクトはENZYMESの時に加えて以下の3つの特性も持っている。

* ```train_mask``` : 訓練用ノードを示すマスク
* ```val_mask```: 評価用ノードを示すマスク
* ```test_mask```: テスト用ノードを示すマスク

### 1.3 ミニバッチ
通常ニューラルネットワークは[ミニバッチで学習する](https://qiita.com/omiita/items/1735c1d048fe5f611f80#4-%E3%83%9F%E3%83%8B%E3%83%90%E3%83%83%E3%83%81%E5%AD%A6%E7%BF%92sgd)ので、ここでPyTorch Geometricによるミニバッチの生成方法を見る。

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print("ミニバッチの情報: ", batch)
    print("ミニバッチ内のグラフの種類数: ", batch.num_graphs)
    x = scatter_mean(batch.x, batch.batch, dim=0)
    print("ミニバッチ内のグラフの種類ごとに平均値をとったときの形状: ", x.size())

    break

ミニバッチの情報:  Batch(batch=[1005], edge_index=[2, 3950], x=[1005, 21], y=[32])
ミニバッチ内のグラフの種類数:  32
ミニバッチ内のグラフの種類ごとに平均値をとったときの形状:  torch.Size([32, 21])


### 1.4 データ加工
PyTorchで画像加工をするときは```torchvision```を使うのが通常のやり方であるが、PyTorch Geometricでは独自の加工メソッドを持つ。
 ```Data```オブジェクトを入力とし、加工後の```Data```オブジェクトを出力とする。
 ```torch_geometric.transforms.Compose```によって複数の処理を一緒くたにすることができる。
 
 例として3次元画像のデータセットShapeNetを使う。
```pre_transform```にはデータセットをディスク上に保存する前に適用する加工を定義する。今回はk-近傍法でグラフを作っている。

In [ ]:
from torch_geometric.datasets import ShapeNet
import torch_geometric.transforms as T

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                   pre_transform=T.KNNGraph(k=6),
                  transform=T.RandomTranslate(0.01)) #各ノードの位置をランダムで少しだけずらす。
dataset[0]

### 1.5 Graph Convolutional Networks
いよいよGCNを使ってみる。Coraデータセットを使用。

#### 1.5.1 データセット

In [ ]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

#### 1.5.2 モデル

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

#### 1.5.3 学習
学習の時の手順は以下の5つをエポック回だけ繰り返す。

1. optimizerの勾配初期化
2. モデルからの予測値を得る
3. 予測値と正解値で損失をとる
4. 損失のパックプロップ
5. optimizerによるパラメータ更新

つまり、
1. **勾配初期化**
2. **予測値**
3. **損失**
4. **バックプロップ**
5. **更新**

で、コードで書けば、
1. ```optimizer.zero_grad()```
2. ```output=model(input)```
3. ```loss=Loss(output, target)```
4. ```loss.backward()```
5. ```optimizer.step()```

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train() #モデルを訓練モードにする。
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

#### 1.5.4 評価

In [ ]:
model.eval() #モデルを評価モードにする。
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Acc: {:.4f}' .format(acc))

Acc: 0.8000
